In [1]:
import numpy as np
import keras

import os
import glob
import cv2
import datetime
import pandas as pd
import time
import warnings

warnings.filterwarnings("ignore")

from sklearn.cross_validation import KFold
from keras.models import Sequential
from keras.layers.core import Dense, Dropout, Flatten
from keras.layers.convolutional import Convolution2D, MaxPooling2D, ZeroPadding2D
from keras.optimizers import SGD
from keras.callbacks import EarlyStopping
from keras.utils import np_utils
from sklearn.metrics import log_loss
from keras import __version__ as keras_version
from keras.callbacks import EarlyStopping

import data_set
%load_ext autoreload
%autoreload 2



Using TensorFlow backend.
//anaconda/lib/python3.5/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [2]:
from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img

In [4]:
FLAGS = dict()
FLAGS['width'] = 32
FLAGS['height'] = 32
FLAGS['batch_size'] = 20
FLAGS['kernel_1_out'] = 8
FLAGS['kernel_2_out'] = 8
FLAGS['conv2_input_width'] = 16
FLAGS['conv2_input_height'] = 16
FLAGS['n_classes'] = 8
FLAGS['learning_rate'] = 0.001
FLAGS['batch_size'] = 16
FLAGS['n_epochs'] = 20
FLAGS['train_report_step'] = 20
FLAGS['val_report_step'] = 80
FLAGS['keep_prob'] = 0.75
FLAGS['reg'] = 0.01
FLAGS['patience'] = 3

In [31]:
data = data_set.DataSet(width = FLAGS['width'], height=FLAGS['height'])


Read train images
Load folder ALB (Index: 0)
Load folder BET (Index: 1)
Load folder DOL (Index: 2)
Load folder LAG (Index: 3)
Load folder NoF (Index: 4)
Load folder OTHER (Index: 5)
Load folder SHARK (Index: 6)
Load folder YFT (Index: 7)
Read train data time: 320.32 seconds


In [32]:
X_val, y_val = data.get_validation_set()

In [58]:
def create_model():
    model = Sequential()
    model.add(ZeroPadding2D((1, 1), input_shape=(32, 32, 3)))
    model.add(Convolution2D(4, 3, 3, activation='relu'))
    model.add(ZeroPadding2D((1, 1)))
    model.add(Convolution2D(4, 3, 3, activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))

    model.add(ZeroPadding2D((1, 1)))
    model.add(Convolution2D(8, 3, 3, activation='relu'))
    model.add(ZeroPadding2D((1, 1)))
    model.add(Convolution2D(8, 3, 3, activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))
    
    model.add(Flatten())
    model.add(Dense(32, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(32, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(8, activation='softmax'))

    sgd = SGD(lr=1e-2, decay=1e-6, momentum=0.9, nesterov=True)
    model.compile(optimizer=sgd, loss='categorical_crossentropy')

    return model


In [59]:
model = create_model()

In [60]:
datagen = ImageDataGenerator(horizontal_flip=True,
                            rotation_range=40,
                            fill_mode='nearest')

In [61]:
val_generator = ImageDataGenerator()
val_gen_flow = val_generator.flow(X_val, y_val, batch_size=FLAGS['batch_size'])

In [63]:
model.fit_generator?

In [62]:
steps_per_epoch = int(data.X_train.shape[0]/FLAGS['batch_size'])
callbacks = [EarlyStopping(monitor='val_loss', patience=5, verbose=0)]
model.fit_generator(datagen.flow(data.X_train, 
                                 data.y_train,
                                 shuffle=True,
                                 batch_size=FLAGS['batch_size']),
                                 samples_per_epoch=data.X_train.shape[0],
                                 nb_epoch=20,
                                 validation_data=val_gen_flow,
                                 nb_val_samples=X_val.shape[0],
                                 callbacks = callbacks)

Epoch 1/20
3021/3021 [==============================] - 13s - loss: 1.6950 - val_loss: 1.6338
Epoch 2/20
3021/3021 [==============================] - 11s - loss: 1.6300 - val_loss: 1.6457
Epoch 3/20
3021/3021 [==============================] - 11s - loss: 1.6240 - val_loss: 1.6405
Epoch 4/20
3021/3021 [==============================] - 12s - loss: 1.6230 - val_loss: 1.6399
Epoch 5/20
3021/3021 [==============================] - 11s - loss: 1.6079 - val_loss: 1.5705
Epoch 6/20
3021/3021 [==============================] - 12s - loss: 1.6127 - val_loss: 1.6324
Epoch 7/20
3021/3021 [==============================] - 11s - loss: 1.6210 - val_loss: 1.6379
Epoch 8/20
3021/3021 [==============================] - 11s - loss: 1.6201 - val_loss: 1.6315
Epoch 9/20
3021/3021 [==============================] - 13s - loss: 1.6201 - val_loss: 1.6368
Epoch 10/20
3021/3021 [==============================] - 15s - loss: 1.6201 - val_loss: 1.6209
Epoch 11/20
3021/3021 [==============================] - 12

In [55]:
batches = 0
for X_batch, y_batch in datagen.flow(data.X_train, 
                                 data.y_train, 
                                 batch_size=FLAGS['batch_size']):
    print(y_batch)
    batches +=1
    if batches > 2:
        break
        

[[ 0.  0.  0.  0.  0.  1.  0.  0.]
 [ 1.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  1.  0.  0.]
 [ 0.  0.  0.  0.  0.  1.  0.  0.]
 [ 1.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  1.]
 [ 1.  0.  0.  0.  0.  0.  0.  0.]
 [ 1.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  1.]
 [ 1.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  1.  0.  0.  0.]
 [ 1.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  1.  0.  0.]
 [ 1.  0.  0.  0.  0.  0.  0.  0.]
 [ 1.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  1.  0.  0.]]
[[ 0.  0.  0.  0.  0.  0.  0.  1.]
 [ 0.  0.  0.  0.  1.  0.  0.  0.]
 [ 1.  0.  0.  0.  0.  0.  0.  0.]
 [ 1.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  1.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  1.  0.  0.  0.]
 [ 1.  0.  0.  0.  0.  0.  0.  0.]
 [ 1.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  1.  0.  0.]
 [ 1.  0.  0.  0.  0.  0.  0.  0.]
 [ 1.  0.  0.  0.  0.  0.  0.  0.]
 [ 1.  0.  0.  0.  0.  0.  0.  0.]
 [ 1.  0.  0.  0.  

In [47]:
import load_images
X_test, y_test = load_images.load_test(width = FLAGS['width'], 
                                                      height=FLAGS['height'])

loading 0 of 1000
loading 20 of 1000
loading 40 of 1000
loading 60 of 1000
loading 80 of 1000
loading 100 of 1000
loading 120 of 1000
loading 140 of 1000
loading 160 of 1000
loading 180 of 1000
loading 200 of 1000
loading 220 of 1000
loading 240 of 1000
loading 260 of 1000
loading 280 of 1000
loading 300 of 1000
loading 320 of 1000
loading 340 of 1000
loading 360 of 1000
loading 380 of 1000
loading 400 of 1000
loading 420 of 1000
loading 440 of 1000
loading 460 of 1000
loading 480 of 1000
loading 500 of 1000
loading 520 of 1000
loading 540 of 1000
loading 560 of 1000
loading 580 of 1000
loading 600 of 1000
loading 620 of 1000
loading 640 of 1000
loading 660 of 1000
loading 680 of 1000
loading 700 of 1000
loading 720 of 1000
loading 740 of 1000
loading 760 of 1000
loading 780 of 1000
loading 800 of 1000
loading 820 of 1000
loading 840 of 1000
loading 860 of 1000
loading 880 of 1000
loading 900 of 1000
loading 920 of 1000
loading 940 of 1000
loading 960 of 1000
loading 980 of 1000


In [48]:
preds = model.predict(X_test)

In [49]:
import make_submission
submit = make_submission.makeSubmission(preds, y_test)

In [50]:
submit.tail(20)

,ALB,BET,DOL,LAG,NoF,OTHER,SHARK,YFT,image
980,0.446644,0.050723,0.031137,0.018144,0.120696,0.082346,0.050472,0.199838,img_07663.jpg
981,0.439816,0.050385,0.032075,0.018345,0.121128,0.082526,0.051914,0.203811,img_07678.jpg
982,0.439934,0.050387,0.032054,0.018341,0.121079,0.082504,0.051890,0.203811,img_07689.jpg
983,0.439816,0.050385,0.032075,0.018345,0.121128,0.082526,0.051914,0.203811,img_07700.jpg
984,0.439886,0.050392,0.032079,0.018342,0.121123,0.082566,0.051918,0.203695,img_07717.jpg
985,0.439816,0.050385,0.032075,0.018345,0.121128,0.082526,0.051914,0.203811,img_07746.jpg
986,0.439760,0.050407,0.032078,0.018347,0.121247,0.082518,0.051895,0.203748,img_07757.jpg
987,0.439816,0.050385,0.032075,0.018345,0.121128,0.082526,0.051914,0.203811,img_07792.jpg
988,0.439816,0.050385,0.032075,0.018345,0.121128,0.082526,0.051914,0.203811,img_07799.jpg
989,0.439816,0.050385,0.032075,0.018345,0.121128,0.082526,0.051914,0.203811,img_07818.jpg


In [18]:
preds = model.predict(X_test, batch_size=FLAGS['batch_size'], verbose=2)


In [24]:
import make_submission
submit = make_submission.makeSubmission(preds,y_test)

In [27]:
submit.to_csv('keras_32_32.csv', index=False)